# Summarizing Text with Amazon Reviews

The objective of this project is to build a model that can create relevant summaries for reviews written about fine foods sold on Amazon. This dataset contains above 500,000 reviews, and is hosted on [Kaggle.](https://www.kaggle.com/snap/amazon-fine-food-reviews)

To build our model we will use a two-layered bidirectional RNN with LSTMs on the input data and two layers, each with an LSTM using bahdanau attention on the target data. [Jaemin Cho's tutorial](https://github.com/j-min/tf_tutorial_plus/tree/master/RNN_seq2seq/contrib_seq2seq) for seq2seq was really helpful to get the code in working order because this is my first project with TensorFlow 1.1; some of the functions are very different from 1.0. The architecture for this model is similar to Xin Pan's and Peter Liu's, here's their [GitHub page.](https://github.com/tensorflow/models/tree/master/textsum)

The sections of this project are:
- Inspecting the Data
- Preparing the Data
- Building the Model
- Training the Model
- Making Our Own Summaries

In [1]:
import pandas as pd
import numpy as np

import re
import time
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

/home/rahul/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.4.1


## Insepcting the Data

In [2]:
import pandas as pd
import json
from pprint import pprint
import glob

# text = []
# title = []
# for file in glob.glob('./entertainment/*.json'):
#     data = json.load(open(file , encoding="utf8"))

#     # map headlines and descriptions to lower case

#     text.append(data['text'].lower())
#     #print(len(data['text'].split(' ')))
#     #print(data['text'])
#     title.append(data['title'].lower())
    

# d = { 'Description' : text , 'Title': title}
# df = pd.DataFrame(d)


In [3]:
df = pd.read_csv('travel.csv' , sep='\t' , encoding='utf-8')


In [4]:
reviews = pd.DataFrame(df)

In [5]:
reviews.shape

(41700, 3)

In [6]:
del reviews['Unnamed: 0']

In [7]:
# Inspecting some of the reviews
for i in range(10,12):
    print("Review #",i+1)
    print(reviews.Summary[i])
    print()
    print(reviews.Text[i])
    print()

Review # 11
Visit California Premieres "Dream of Dance" - an Inside Look at Choreographer Ryan Heffington's Latest Music Video "Anna"

 Visit California
Premiere event held at GRAMMY Museum with Will Butler, Emma Stone & Ryan Heffington
LOS ANGELES , Oct. 15, Visit California hosted an exclusive panel discussion last night with the cast and crew of "Anna," a new single and music video by Arcade Fire's Will Butler featuring a dream partnership with actress Emma Stone and world-renowned choreographer Ryan Heffington . Alongside this panel discussion was the premiere screening of Visit California's latest California Dreamers video "Dream of Dance," which explores Heffington's creative world and Golden State inspiration.
At the GRAMMY Museum in Downtown Los Angeles , Caroline Beteta , President and CEO of Visit California, led a conversation with "Anna" talent and crew. Emma Stone discussed her music video debut, while Heffington noted how his work has been inspired by California icons suc

## Preparing the Data

In [8]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [9]:
import nltk
from nltk.corpus import stopwords

In [10]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r]', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    n_t = text.split()
    n_t = n_t[:120]
    
    return " ".join(n_t)

We will remove the stopwords from the texts because they do not provide much use for training our model. However, we will keep them for our summaries so that they sound more like natural phrases. 

In [11]:
# Clean the summaries and texts
sss = 0

clean_summaries = []
for summary in reviews.Summary:
    

        
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
    sss +=1 
    if sss==7501:
        break
    #lean_summaries.append(summary)
    
print("Summaries are complete.")

sss = 0
clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
    sss +=1 
    if sss==7501:
        break
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [12]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(10,15):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print()
    print(clean_texts[i])
    print()

Clean Review # 11
visit california premieres dream of dance an inside look at choreographer ryan heffington s latest music video anna

visit california premiere event held grammy museum butler emma stone ryan heffington los angeles oct 15 visit california hosted exclusive panel discussion last night cast crew anna new single music video arcade fire butler featuring dream partnership actress emma stone world renowned choreographer ryan heffington alongside panel discussion premiere screening visit california latest california dreamers video dream dance explores heffington creative world golden state inspiration grammy museum downtown los angeles caroline beteta president ceo visit california led conversation anna talent crew emma stone discussed music video debut heffington noted work inspired california icons queen mary long beach key centerpiece video dream dance explores heffington california roots creative process along behind scenes footage anna work includes acclaimed sia video ch

In [13]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [14]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 53377


In [15]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('./numberbatch-en.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

    
print('Word embeddings:', len(embeddings_index))


Word embeddings: 417195


In [16]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 10
ss = []
for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            ss.append(word)
            missing_words += 1
            
missing_percentage = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_percentage))



Number of words missing from CN: 735
Percent of words that are missing from vocabulary: 1.38%


I use a threshold of 20, so that words not in CN can be added to our word_embedding_matrix, but they need to be common enough in the reviews so that the model can understand their meaning.

In [17]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= 2 or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_percentage = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_percentage))

Total number of unique words: 53377
Number of words we will use: 41518
Percent of words we will use: 77.78%


In [18]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

41518


In [19]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [20]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 942268
Total number of UNKs in headlines: 11863
Percent of words that are UNK: 1.26%


In [21]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [22]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
            counts
count  7501.000000
mean     10.205439
std       3.857709
min       1.000000
25%       8.000000
50%      10.000000
75%      12.000000
max      71.000000

Texts:
            counts
count  7501.000000
mean    116.413545
std      14.801059
min      26.000000
25%     121.000000
50%     121.000000
75%     121.000000
max     121.000000


In [23]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 60))
print(np.percentile(lengths_texts.counts, 75))
print(np.percentile(lengths_texts.counts, 90))

121.0
121.0
121.0


In [24]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 60))
print(np.percentile(lengths_summaries.counts, 75))
print(np.percentile(lengths_summaries.counts, 90))

10.0
12.0
15.0


In [25]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [26]:
# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

# sorted_summaries = []
# sorted_texts = []
# max_text_length = 600
# max_summary_length = 15
# min_slength = 1
# min_tlength = 100
# unk_text_limit = 8
# unk_summary_limit = 0

 
# for count, words in enumerate(int_summaries):
#     if (len(int_summaries[count]) >= min_slength and
#             len(int_summaries[count]) <= max_summary_length and
#             len(int_texts[count]) >= min_tlength and
#             unk_counter(int_summaries[count]) <= unk_summary_limit and
#             unk_counter(int_texts[count]) <= unk_text_limit
#            ):
#         sorted_summaries.append(int_summaries[count])
#         sorted_texts.append(int_texts[count])
        
# # Compare lengths to ensure they match
# print(len(sorted_summaries))
# print(len(sorted_texts))

sorted_summaries = []
sorted_texts = []
max_text_length = 122
max_summary_length = 15
min_length = 2

unk_text_limit = 5
unk_summary_limit = 0
for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

6079
6079


## Building the Model

In [27]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [28]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [29]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [30]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits,_,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [31]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [32]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell, attn_mech, rnn_size)
            
    initial_state = dec_cell.zero_state(dtype=tf.float32 , batch_size=batch_size)
    
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)

    return training_logits, inference_logits

In [33]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [34]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [35]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [36]:
# Set the Hyperparameters
epochs = 60
batch_size = 64  
rnn_size = 256
num_layers = 3
learning_rate = 0.005
keep_probability = 0.9

In [37]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Graph is built.


In [38]:
# writer = tf.summary.FileWriter("./ss")
# writer.add_graph(train_graph)

## Training the Model

Since I am training this model on my MacBook Pro, it would take me days if I used the whole dataset. For this reason, I am only going to use a subset of the data, so that I can train it over night. Normally I use [FloydHub's](https://www.floydhub.com/) services for my GPU needs, but it would take quite a bit of time to upload the dataset and ConceptNet Numberbatch, so I'm not going to bother with that for this project.

I chose not use use the start of the subset because I didn't want to make it too easy for my model. The texts that I am using are closer to the median lengths; I thought this would be more fair.

In [39]:
# Subset the data for training
start = 0
end = -1
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))




The shortest text length: 30
The longest text length: 121


In [40]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 25 # If the update loss does not decrease in 25 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "./best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #     loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #     loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    #if stop_early == stop:
                    #   break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
#         if stop_early == stop:
#             print("Stopping Training.")
#             break

Epoch   1/60 Batch   20/94 - Loss:  7.179, Seconds: 44.01
Average loss for this update: 6.609
New Record!
Epoch   1/60 Batch   40/94 - Loss:  5.417, Seconds: 44.19
Epoch   1/60 Batch   60/94 - Loss:  5.313, Seconds: 45.53
Average loss for this update: 5.33
New Record!
Epoch   1/60 Batch   80/94 - Loss:  5.052, Seconds: 44.07
Average loss for this update: 5.039
New Record!
Epoch   2/60 Batch   20/94 - Loss:  4.638, Seconds: 45.40
Average loss for this update: 4.591
New Record!
Epoch   2/60 Batch   40/94 - Loss:  4.467, Seconds: 45.27
Epoch   2/60 Batch   60/94 - Loss:  4.448, Seconds: 44.63
Average loss for this update: 4.445
New Record!
Epoch   2/60 Batch   80/94 - Loss:  4.334, Seconds: 44.00
Average loss for this update: 4.338
New Record!
Epoch   3/60 Batch   20/94 - Loss:  4.200, Seconds: 43.99
Average loss for this update: 4.156
New Record!
Epoch   3/60 Batch   40/94 - Loss:  4.036, Seconds: 44.95
Epoch   3/60 Batch   60/94 - Loss:  3.992, Seconds: 44.10
Average loss for this updat

Epoch  22/60 Batch   80/94 - Loss:  0.749, Seconds: 43.96
Average loss for this update: 0.744
New Record!
Epoch  23/60 Batch   20/94 - Loss:  0.707, Seconds: 43.95
Average loss for this update: 0.721
New Record!
Epoch  23/60 Batch   40/94 - Loss:  0.740, Seconds: 43.90
Epoch  23/60 Batch   60/94 - Loss:  0.710, Seconds: 44.09
Average loss for this update: 0.717
New Record!
Epoch  23/60 Batch   80/94 - Loss:  0.712, Seconds: 43.90
Average loss for this update: 0.702
New Record!
Epoch  24/60 Batch   20/94 - Loss:  0.661, Seconds: 44.01
Average loss for this update: 0.666
New Record!
Epoch  24/60 Batch   40/94 - Loss:  0.681, Seconds: 44.09
Epoch  24/60 Batch   60/94 - Loss:  0.673, Seconds: 44.24
Average loss for this update: 0.677
No Improvement.
Epoch  24/60 Batch   80/94 - Loss:  0.673, Seconds: 44.00
Average loss for this update: 0.672
No Improvement.
Epoch  25/60 Batch   20/94 - Loss:  0.642, Seconds: 44.19
Average loss for this update: 0.652
New Record!
Epoch  25/60 Batch   40/94 -

Epoch  44/60 Batch   40/94 - Loss:  0.280, Seconds: 43.81
Epoch  44/60 Batch   60/94 - Loss:  0.295, Seconds: 44.06
Average loss for this update: 0.291
No Improvement.
Epoch  44/60 Batch   80/94 - Loss:  0.294, Seconds: 43.77
Average loss for this update: 0.291
No Improvement.
Epoch  45/60 Batch   20/94 - Loss:  0.265, Seconds: 44.00
Average loss for this update: 0.267
New Record!
Epoch  45/60 Batch   40/94 - Loss:  0.281, Seconds: 44.06
Epoch  45/60 Batch   60/94 - Loss:  0.289, Seconds: 44.14
Average loss for this update: 0.29
No Improvement.
Epoch  45/60 Batch   80/94 - Loss:  0.286, Seconds: 44.04
Average loss for this update: 0.28
No Improvement.
Epoch  46/60 Batch   20/94 - Loss:  0.264, Seconds: 43.98
Average loss for this update: 0.272
No Improvement.
Epoch  46/60 Batch   40/94 - Loss:  0.281, Seconds: 43.98
Epoch  46/60 Batch   60/94 - Loss:  0.275, Seconds: 44.09
Average loss for this update: 0.274
No Improvement.
Epoch  46/60 Batch   80/94 - Loss:  0.474, Seconds: 43.94
Aver

KeyboardInterrupt: 

## Making Our Own Summaries

To see the quality of the summaries that this model can generate, you can either create your own review, or use a review from the dataset. You can set the length of the summary to a fixed value, or use a random value like I have here.

In [41]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [43]:
o_text = []
o_sum = []
dl_sum = []

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()


with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    for random in range(len(clean_texts)):
        #print(clean_summaries[random])
        o_sum.append(clean_summaries[random])
        input_sentence = clean_texts[random]
        o_text.append(input_sentence)
        z = input_sentence.split()[:120]
        input_sentence = " ".join(z)
        text = text_to_seq(clean_texts[random])
        #text = text_to_seq(input_sentence)



        #Multiply by batch_size to match the model's input parameters
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(8,13)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

        # Remove the padding from the tweet
        pad = vocab_to_int["<PAD>"] 

        #print('Original Text:', input_sentence)
        
        

        #print('\nText')
        #print('  Word Ids:    {}'.format([i for i in text]))
        #print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

        #print('\nSummary')
        #print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
        nn = " ".join([int_to_vocab[i] for i in answer_logits if i != pad])
        dl_sum.append(nn)

INFO:tensorflow:Restoring parameters from ./best_model.ckpt


In [44]:
dic = { 'Original_Summary' : o_sum , 'Generated_Summry' : dl_sum , 'Original_text' : o_text}
dic = pd.DataFrame(dic)
dic.to_csv('Output_5.csv')

In [45]:
dic.shape

(7501, 3)

Examples of reviews and summaries:
- Review(1): The coffee tasted great and was at such a good price! I highly recommend this to everyone!
- Summary(1): great coffee


- Review(2): This is the worst cheese that I have ever bought! I will never buy it again and I hope you won't either!
- Summary(2): omg gross gross


- Review(3): love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets
- Summary(3): love it

## Summary

I hope that you found this project to be rather interesting and informative. One of my main recommendations for working with this dataset and model is either use a GPU, a subset of the dataset, or plenty of time to train your model. As you might be able to expect, the model will not be able to make good predictions just by seeing many reviews, it needs so see the reviews many times to be able to understand the relationship between words and between descriptions & summaries. 

In short, I'm pleased with how well this model performs. After creating numerous reviews and checking those from the dataset, I can happily say that most of the generated summaries are appropriate, some of them are great, and some of them make mistakes. I'll try to improve this model and if it gets better, I'll update my GitHub.

Thanks for reading!